In [1]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [2]:
file_path = '../data/adult.csv'
df = pd.read_csv(file_path)

df.replace('?', pd.NA, inplace=True)
df.dropna(inplace=True)

# Task 1
Print the count of men and women in the dataset.

In [4]:
gender_counts = df['sex'].value_counts()
print(gender_counts)

sex
Male      20380
Female     9782
Name: count, dtype: int64


# Task 2
Find the average age of men in dataset

In [5]:
average_age_men = df[df['sex'] == 'Male']['age'].mean()
print(average_age_men)

39.18400392541707


# Task 3
Get the percentage of people from Poland (native-country)

In [8]:
poland_percentage = (df['native-country'] == 'Poland').mean() * 100
print(poland_percentage)

0.18566408063125786


# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [10]:
high_income_age = df[df['salary'] == '>50K']['age'].agg(['mean', 'std'])
low_income_age = df[df['salary'] == '<=50K']['age'].agg(['mean', 'std'])
print("High income age stats:", high_income_age)
print("Low income age stats:", low_income_age)

High income age stats: mean    43.959110
std     10.269633
Name: age, dtype: float64
Low income age stats: mean    36.608060
std     13.464631
Name: age, dtype: float64


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [13]:
higher_education = ['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate']
high_income_no_higher_edu = df[(df['salary'] == '>50K') & (~df['education'].isin(higher_education))]
print(high_income_no_higher_edu.shape[0])

3178


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [14]:
age_stats_by_education = df.groupby('education')['age'].describe()
print(age_stats_by_education)

               count       mean        std   min   25%   50%   75%   max
education                                                               
10th           820.0  37.897561  16.225795  17.0  23.0  36.0  52.0  90.0
11th          1048.0  32.363550  15.089307  17.0  18.0  28.5  43.0  90.0
12th           377.0  32.013263  14.373710  17.0  19.0  28.0  41.0  79.0
1st-4th        151.0  44.622517  14.929051  19.0  33.0  44.0  56.0  81.0
5th-6th        288.0  41.649306  14.754622  17.0  28.0  41.0  53.0  82.0
7th-8th        557.0  47.631957  15.737479  17.0  34.0  49.0  60.0  90.0
9th            455.0  40.303297  15.335754  17.0  28.0  38.0  53.0  90.0
Assoc-acdm    1008.0  37.286706  10.509755  19.0  29.0  36.0  44.0  90.0
Assoc-voc     1307.0  38.246366  11.181253  19.0  30.0  37.0  45.0  84.0
Bachelors     5044.0  38.641554  11.577566  19.0  29.0  37.0  46.0  90.0
Doctorate      375.0  47.130667  11.471727  24.0  39.0  47.0  54.0  80.0
HS-grad       9840.0  38.640955  13.067730  17.0  2

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [15]:
df['is_married'] = df['marital-status'].str.startswith('Married')
salary_by_marital_status = df[df['sex'] == 'Male'].groupby(['is_married', 'salary']).size().unstack(fill_value=0)
print(salary_by_marital_status)

salary      <=50K  >50K
is_married             
False        6932   673
True         7052  5723


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [16]:
max_hours_per_week = df['hours-per-week'].max()
people_working_max_hours = df[df['hours-per-week'] == max_hours_per_week].shape[0]
print("Max hours per week:", max_hours_per_week)
print("People working max hours per week:", people_working_max_hours)

Max hours per week: 99
People working max hours per week: 78


# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [18]:
numerical_df = df.select_dtypes(include=['int64', 'float64'])
correlation_matrix = numerical_df.corr()
print(correlation_matrix)

                Unnamed: 0       age  hours-per-week  salary K$
Unnamed: 0        1.000000 -0.001126       -0.001890   0.000129
age              -0.001126  1.000000        0.101599   0.208203
hours-per-week   -0.001890  0.101599        1.000000   0.196378
salary K$         0.000129  0.208203        0.196378   1.000000
